In [4]:
import numpy as np
import gc
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

# Data pre-prep; Don't run again

In [3]:
df1 = pd.read_csv('data/csgo_price_hist.csv')
df2 = pd.read_csv('data/csgo_price_hist2.csv')

In [4]:
duplicate_check = df1.merge(df2, on=list(df1.columns), how="inner")
if duplicate_check.shape[0] == 0:
  print('No duplicates across df1 and df2')

No duplicates across df1 and df2


In [6]:
def agg_hist_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Group the dataframe by 'market', 'market_hash_name', and 'dates.date'.
    Sums the 'quantity' and takes the minimum of 'price'.

    Parameters:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The aggregated DataFrame.
    """
    return df.groupby(['market_hash_name', 'dates.date']).agg({
        'quantity': 'sum',
        'price': 'min'
    }).reset_index()


agg_df1 = agg_hist_data(df1)
agg_df2 = agg_hist_data(df2)

print(agg_df1.shape)
print(agg_df2.shape)

(192366, 4)
(7132894, 4)


In [9]:
del df1
del df2
gc.collect()

In [10]:
agg_df_hist = pd.concat([agg_df1, agg_df2])
agg_df_hist.to_csv('data/csgo_agg_df_hist.csv')
agg_df_hist.shape

(7325260, 4)

In [12]:
agg_df_hist.sort_values('dates.date', ascending=False).head()

,market_hash_name,dates.date,quantity,price
7132893,★ Ursus Knife | Urban Masked (Well-Worn),2023-09-19,50,12593
1343707,Pinups Capsule,2023-09-19,2359,81
2747607,StatTrak™ AK-47 | Rat Rod (Battle-Scarred),2023-09-19,247,762
5936742,USP-S | Guardian (Minimal Wear),2023-09-19,1833,339
1344071,Poorly Drawn Capsule,2023-09-19,722,80


In [13]:
agg_df_hist = pd.read_csv('data/csgo_agg_df_hist.csv', index_col=0)
agg_df_hist.price = agg_df_hist.price/100  # Get data in USD (isntead of USD cents)
agg_df_hist = agg_df_hist.loc[(agg_df_hist['dates.date'] > '2022-09-20') & 
                              (agg_df_hist['dates.date'] < '2023-09-19'), :]

In [14]:
fluctuation = agg_df_hist.groupby('market_hash_name').agg({
    'quantity': ['std', 'mean', 'min', 'max',
                 lambda x: x.quantile(0.10),
                 lambda x: x.quantile(0.25),
                 lambda x: x.quantile(0.50),
                 lambda x: x.quantile(0.75),
                 lambda x: x.quantile(0.90)],
    'price': ['mean']
}).reset_index()

fluctuation.columns = [
    'market_hash_name',
    'std_quantity',
    'mean_quantity',
    'min_quantity',
    'max_quantity',
    '10th_percentile_quantity',
    '25th_percentile_quantity',
    'median',
    '75th_percentile_quantity',
    '90th_percentile_quantity',
    'mean_price'
]

fluctuation['coef_of_variation'] = fluctuation['std_quantity'] / fluctuation['mean_quantity']
fluctuation.head()


,market_hash_name,std_quantity,mean_quantity,min_quantity,max_quantity,10th_percentile_quantity,25th_percentile_quantity,median,75th_percentile_quantity,90th_percentile_quantity,mean_price,coef_of_variation
0,'Medium Rare' Crasswater | Guerrilla Warfare,306.448272,1000.168044,236,1589,674.0,771.0,908.0,1313.5,1428.8,8.491983,0.306397
1,'The Doctor' Romanov | Sabre,301.795136,3281.994490,2025,3901,2889.8,3077.0,3268.0,3503.5,3711.0,3.829394,0.091955
2,'Two Times' McCoy | TACP Cavalry,448.358181,1454.044077,711,2445,904.0,1095.5,1340.0,1831.5,2111.8,1.686722,0.308353
3,'Two Times' McCoy | USAF TACP,574.310914,1993.162534,698,3231,1267.4,1546.5,1866.0,2465.0,2799.0,1.639339,0.288141
4,10 Year Birthday Sticker Capsule,2651.694205,5343.300275,1260,12531,2089.4,3294.0,4782.0,7224.5,9408.6,0.768457,0.496265


In [15]:
agg_df_hist = agg_df_hist.merge(fluctuation, on='market_hash_name', how='inner')
agg_df_hist.to_csv('data/csgo_agg_df_hist_fluct.csv')

# Start here

In [16]:
agg_df_hist = pd.read_csv('data/csgo_agg_df_hist_fluct.csv', index_col=0)
print('shape: ', agg_df_hist.shape)
agg_df_hist.head()

shape:  (7304522, 15)


,market_hash_name,dates.date,quantity,price,std_quantity,mean_quantity,min_quantity,max_quantity,10th_percentile_quantity,25th_percentile_quantity,median,75th_percentile_quantity,90th_percentile_quantity,mean_price,coef_of_variation
0,10 Year Birthday Sticker Capsule,2022-09-21,3620,0.74,2651.694205,5343.300275,1260,12531,2089.4,3294.0,4782.0,7224.5,9408.6,0.768457,0.496265
1,10 Year Birthday Sticker Capsule,2022-09-22,3465,0.73,2651.694205,5343.300275,1260,12531,2089.4,3294.0,4782.0,7224.5,9408.6,0.768457,0.496265
2,10 Year Birthday Sticker Capsule,2022-09-23,3364,0.73,2651.694205,5343.300275,1260,12531,2089.4,3294.0,4782.0,7224.5,9408.6,0.768457,0.496265
3,10 Year Birthday Sticker Capsule,2022-09-24,2906,0.73,2651.694205,5343.300275,1260,12531,2089.4,3294.0,4782.0,7224.5,9408.6,0.768457,0.496265
4,10 Year Birthday Sticker Capsule,2022-09-25,2460,0.74,2651.694205,5343.300275,1260,12531,2089.4,3294.0,4782.0,7224.5,9408.6,0.768457,0.496265


# Useful functions

In [17]:
def construct_index(df):
    # Convert the 'dates.date' column to datetime
    df['dates.date'] = pd.to_datetime(df['dates.date'])

    # Calculate the index for each day
    df['index'] = df['quantity'] * df['price']

    # Group by date and sum the indices
    index_data = df.groupby('dates.date')['index'].sum().reset_index()

    return index_data


In [18]:
def plot_indexes(index1, index2, title):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=index1['dates.date'], y=index1['index'], name='Index 1', mode='lines'))

    fig.add_trace(go.Scatter(x=index2['dates.date'], y=index2['index'], name='Index 2', mode='lines'))

    fig.update_layout(
        title=title,
        xaxis_title='dates',
        yaxis_title='index'
    )

    fig.show()

In [19]:
raw_index = construct_index(agg_df_hist)

fig = px.line(raw_index, x='dates.date', y='index', title='Raw CS:GO Skin Index')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Index Value')
fig.show()

In [20]:
def most_traded_index(df, n, quantity_measure, raw_index=raw_index,
                      compare=True, min_quantity=0, plot=True):

  # Supported quantity_measure values:
  # "mean_quantity", "min_quantity", "max_quantity", "10th_percentile_quantity",
  # "25th_percentile_quantity", "median",	"75th_percentile_quantity",
  # "90th_percentile_quantity"

  # Calculate number of unique price points for each market_hash_name
  unique_prices = df.groupby('market_hash_name')['price'].nunique().reset_index()
  unique_prices = unique_prices.rename(columns={'price': 'unique_prices'})

  df = df.merge(unique_prices, on='market_hash_name', how='left')
  
  traded_ranking = df[['market_hash_name', 'unique_prices']]\
        .drop_duplicates()\
        .sort_values('unique_prices', ascending=False)
  
  most_traded = df[['market_hash_name', 'dates.date', 'price', quantity_measure]]
  most_traded = most_traded.rename(columns={quantity_measure: 'quantity'})
  top_n_mt = list(traded_ranking[:n]['market_hash_name'])
  most_traded = most_traded[most_traded['market_hash_name'].isin(top_n_mt)]
  most_traded = most_traded[most_traded.quantity >= min_quantity]
  mt_n_index = construct_index(most_traded)
  if plot:
    if compare:
      plot_indexes(raw_index, mt_n_index, f'Most traded {n} (min quantity {min_quantity}) vs raw')
    else:
      fig = px.line(mt_n_index, x='dates.date', y='index', title=f'Most traded {n} (min quantity {min_quantity})')
      fig.update_xaxes(title_text='Date')
      fig.update_yaxes(title_text='Index Value')
      fig.show()
  print(f'Number of products: {most_traded.market_hash_name.drop_duplicates().count()}')
  return mt_n_index, most_traded

In [21]:
def simulate_attack(df, item_list, day_count, increase=True):
  # Start from the 10th date in the data. It could be anything.
  date_list = [df['dates.date'][10] + pd.Timedelta(days=i) for i in range(day_count)]
  # Days and items to target
  condition = ((df['market_hash_name'].isin(item_list)) & (df['dates.date'].isin(date_list)))
  if increase:
    # Attack takes the combined market share of selected items to 99% for the duration of the attack
    df['index_share'] = np.where(condition, 0.99/len(item_list), df['index_share'])
  else:
    # Attack makes the index value of the input items 0
    df['index_share'] = np.where(condition, 1e-10, df['index_share'])
    # df['index'] = np.where(condition, 0, df['index'])
  return df

In [22]:
def get_avg_share(df):
    # GET INDEX SHARE
    daily_sum = df.groupby('dates.date')['index'].sum().reset_index()
    daily_sum = daily_sum.rename(columns={'index': 'daily_sum_index'})
    df = df.merge(daily_sum, on='dates.date')
    df['index_share'] = (df['index'] / df['daily_sum_index'])
    df = df.drop(columns=['daily_sum_index'])

    # GET MAPPING
    half_year_date = df.loc[:, "dates.date"].drop_duplicates().iloc[182,]
    avg_mapping = df.loc[df['dates.date'] < half_year_date, ['market_hash_name', 'dates.date', 'index_share']]

    # Compute mean and standard deviation of index_share by market_hash_name
    avg_mapping = avg_mapping.groupby('market_hash_name')['index_share'].agg(['mean', 'std']).reset_index()
    avg_mapping = avg_mapping.rename(columns={'mean': 'avg_index_share', 'std': 'std_index_share'})

    df = df[df['dates.date'] > half_year_date]
    df = df.merge(avg_mapping, on='market_hash_name', how='inner')
    return df, avg_mapping

In [23]:
# def get_caps(mapping):
#   mapping['cap_index_share'] = np.where(mapping['avg_index_share'] < 0.005, 0.01,
#                                             np.where(mapping['avg_index_share'] < 0.01, 0.015,
#                                                     np.where(mapping['avg_index_share'] < 0.02, 0.025,
#                                                               np.where(mapping['avg_index_share'] < 0.03, 0.032,
#                                                                       np.where(mapping['avg_index_share'] < 0.04, 0.041,
#                                                                                 np.where(mapping['avg_index_share'] < 0.05, 0.05, mapping['avg_index_share']
#                                                                                         )
#                                                                                 )
#                                                                       )
#                                                               )
#                                                     )
#                                             )
#   return mapping

def get_caps(mapping):
  upper_multiplier = 1
  lower_multiplier = 1
  mapping['upper_cap_index_share'] = mapping['avg_index_share'] + upper_multiplier*mapping['std_index_share']
  mapping['lower_cap_index_share'] = np.where(mapping['avg_index_share'] - lower_multiplier*mapping['std_index_share'] > 0,
                                              mapping['avg_index_share'] - lower_multiplier*mapping['std_index_share'], 0)
  return mapping

In [24]:
def cap_index(df):
  valid = df[(df['index_share'] >= df['lower_cap_index_share']) & 
             (df['index_share'] <= df['upper_cap_index_share'])]
  valid_ind = pd.DataFrame(valid.groupby('dates.date')['index'].sum()).reset_index().rename(columns={'index': 'sum_index'})
  
  # Invalid Entries: Those below lower_cap_index_share
  invalid_lower = df[df['index_share'] < df['lower_cap_index_share']]
  invalid_lower_ind = pd.DataFrame(invalid_lower.groupby('dates.date')['lower_cap_index_share'].sum()).reset_index().rename(columns={'lower_cap_index_share': 'sum_lower_cap_index_share'})

  # Invalid Entries: Those above upper_cap_index_share
  invalid_upper = df[df['index_share'] > df['upper_cap_index_share']]
  invalid_upper_ind = pd.DataFrame(invalid_upper.groupby('dates.date')['upper_cap_index_share'].sum()).reset_index().rename(columns={'upper_cap_index_share': 'sum_upper_cap_index_share'})

  # Merging
  all_ind = valid_ind.merge(invalid_lower_ind, on=['dates.date'], how='left').merge(invalid_upper_ind, on=['dates.date'], how='left')
  all_ind = all_ind.fillna({'sum_lower_cap_index_share': 0, 'sum_upper_cap_index_share': 0})

  # Calculating the adjusted index 
  all_ind['index'] = all_ind['sum_index'] / (1 - all_ind['sum_lower_cap_index_share'] - all_ind['sum_upper_cap_index_share'])

  return all_ind



In [25]:
def plot_inds(df1, df2, name1, name2):
  fig = go.Figure()

  fig.add_trace(go.Scatter(x=df2['dates.date'], y=df2['index'], name=name2, mode='lines'))

  fig.add_trace(go.Scatter(x=df1['dates.date'], y=df1['index'], name=name1, mode='lines'))

  fig.update_layout(
      title=f'{name1} vs {name2}',
      xaxis_title='dates',
      yaxis_title='index'
  )

  fig.show()

In [26]:
ind, ind_data = most_traded_index(
    df=agg_df_hist,
    n=5000,
    quantity_measure='min_quantity',
    raw_index=raw_index,
    compare=False,
    min_quantity=100,
    plot=False
)
ind_data, mapping = get_avg_share(ind_data)
map_df = get_caps(ind_data)
capped_ind = cap_index(map_df)
plot_inds(ind, capped_ind, 'Raw Index', 'Capped Index')

Number of products: 550


In [30]:
# mapping.to_csv('data/csgo_mapping.csv')

In [27]:
# # Small items
# item_list = ['Sticker | FalleN (Holo) | Antwerp 2022',
#              'Souvenir AUG | Carved Jade (Factory New)',
#              'Sticker | Astralis (Foil) | Stockholm 2021',
#              'Sticker | 9z Team (Holo) | Antwerp 2022'
# ]

# Big items
item_list = ['AK-47 | Vulcan (Minimal Wear)',
             '★ Karambit | Doppler (Factory New)',
             '★ M9 Bayonet | Fade (Factory New)',
            #  '★ Talon Knife | Doppler (Factory New)'
             ]

In [28]:
ind, ind_data = most_traded_index(
    df=agg_df_hist,
    n=5000,
    quantity_measure='min_quantity',
    raw_index=raw_index,
    compare=False,
    min_quantity=100,
    plot=False
)
ind_data, mapping = get_avg_share(ind_data)
map_df = get_caps(ind_data)
attacked_df = simulate_attack(map_df, item_list, 50, increase=True)
att_capped_ind = cap_index(attacked_df)
plot_inds(capped_ind, att_capped_ind, 'Capped Index', 'Attacked Capped Index')

Number of products: 550
